# Preparación de libraries y funciones.

## Imports.

In [69]:
import numpy as np
import pandas as pd
import GetOldTweets3 as got
import datetime
from textblob import TextBlob
import re
import unidecode
import matplotlib.pyplot as plt
import time
import os
import glob


## Definición de funciones.
En esta sección definimos las funciones que vamos a utilizar en la notebook.

### Función para obtener los tweets de acuerdo a los criterios de búsqueda.

In [70]:
def get_tweets(list_criterios, fecha_inicio, fecha_final, maximo=250):
    tweets_total = []
    for criterio in list_criterios:
        tweetCriteria = got.manager.TweetCriteria().setQuerySearch(criterio).setSince(fecha_inicio).setUntil(fecha_final).setMaxTweets(maximo)
        tweets_total = got.manager.TweetManager.getTweets(tweetCriteria)
    return tweets_total

### Función para limpiar los tweets.

In [71]:
def limpiar(tweet_texto):
    text = ' '.join(re.sub(r"(@)|([^0-9A-Za-z \t])|(www.[^ ]+)|(https?://[^ ]+)", "", unidecode.unidecode(tweet_texto.lower())).split())
    return text

### Función para crear un dataframe en base a los tweets.

In [72]:
def create_df(tweets_anuncios):
    datos = []
    for anuncio in tweets_anuncios:
        for ciudad in tweets_anuncios[anuncio]: 
            for tweet in tweets_anuncios[anuncio][ciudad]:
                registro = {'username': tweet.username, 'tweet': limpiar(tweet.text), \
                            'fecha': tweet.date, 'anuncio':anuncio, 'ubicacion':ciudad, 'id':tweet.id}
                datos.append(registro)
    df = pd.DataFrame(data=datos)    
    return df

### Función para limpiar dataframe.

In [73]:
#definimos listas de los usuarios y topics que vamos a eliminar
usuarios_blacklist = ['sergistack', 'GamesRedDeer']
usuarios_medios = ['gustavorearte1','gerdellamonica','radiofonica1007',' pmgcharly',' NTodxs',' Noticiasde_',' nora_verges',' mnspezzapria',' METRO_RADIO_TV',' MartinD50004804',' mariogaloppo',' LUIS20GEREZ',' losprimerostuc',' lacriticaok',' lacapital',' IldefonsoM',' HernanMundo',' elsolquilmes',' ellitoral',' cronica',' cordoba',' Contexto_Tuc',' ConLaGenteRos',' con_sello',' Cadena3Com',' AvellanedaReal',' AiredeSantaFe',' AgenciaDib',' Adry1BC',' ADNsur',' abccordoba',' 104Urbana']
topicos_excluidos = ['pique','barcelona', 'bayern munich', 'barca', 'bayern', 'barsa', 'anabelle', 'annabell', 'annabelle', 'messi', 'balvin', 'pampita']

def clean_up_df(df):
    #definimos una máscara para excluir a los usuarios en la lista negra
    mask_not_bl = [x not in usuarios_blacklist for x in df['username']] 
    df = df[mask_not_bl]
    #definimos una máscara para excluir a los usuarios en la lista negra
    mask_not_media = [x not in usuarios_medios for x in df['username']] 
    df = df[mask_not_media]
    df.drop_duplicates(subset=['tweet'], keep=False, inplace=True)
    # máscara de tópicos excluidos
    mask_not_topics = [x not in topicos_excluidos for x in df['tweet']] 
    df = df[mask_not_topics]
    
    try:
        df.drop(columns=['index'], axis=0, inplace=True)
    except:
        print("No existe la columna index")
        
    try:
        df.drop(columns=['fecha_sola'], axis=0, inplace=True)
    except:
        print("No existe la columna fecha_sola")
        
    return df

### Datasets con archivos sucios.

Obtenemos todos los archivos crudos limpios y los conformamos en un único dataframe:

In [89]:
path_crudos = 'Data/Crudos/' 
all_files_crudos = glob.glob(path_crudos + "/*.csv")

li_crudos = []

try:
    for filename in all_files_crudos:
        df = pd.read_csv(filename, sep = ';', header=0)
        li_crudos.append(df_sin_clasificar)
    concatenado = pd.concat(li_crudos, axis=0) 
    df = concatenado.drop_duplicates()
    print(df.shape)
except:
    print("No hay archivos en el directorio")

(13857, 8)


In [90]:
print(df.shape)
df.head(3)

(13857, 8)


,username,tweet,fecha,anuncio,ubicacion,id,sentimiento,fecha_sola
0,Elizabeth789741,alferdez alferdezprensa por favor qxsea a partir del sabafo la cuarentena los q fuimos hoy a tra...,2020-03-19 23:04:48+00:00,Anuncio_1,"Buenos Aires, Argentina",1.240776e+18,NaN,NaN
1,sergistack,usas google chrome lo siento por las mayusculas pero existe un nuevo navegador que es mas rapido...,2020-03-28 20:07:18+00:00,Anuncio_1,"Buenos Aires, Argentina",1.243993e+18,NaN,NaN
2,vickyuliyapo,comodice inesazpelicueta que comiencen ya a elaborar remedios genericos asi no nos toma de sorpr...,2020-03-19 22:57:33+00:00,Anuncio_1,"Buenos Aires, Argentina",1.240774e+18,NaN,NaN


# Limpieza.

Tamaño del dataset previo a la limpieza:

In [91]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_colwidth', 100)

### Por usuarios específicos.

#### Trolls.

In [92]:
mask_bl = [x in usuarios_blacklist for x in df['username']] 

In [93]:
df.loc[mask_bl, ['username','tweet']]

,username,tweet
1,sergistack,usas google chrome lo siento por las mayusculas pero existe un nuevo navegador que es mas rapido...
34,sergistack,usas google chrome lo siento por las mayusculas pero existe un nuevo navegador que es mas rapido...
10,sergistack,usas google chrome lo siento por las mayusculas pero existe un nuevo navegador que es mas rapido...
34,GamesRedDeer,waiting for fallguys on nintendo try now joy of clumsyrush cosy party game fallguysgame nintendo...
42,GamesRedDeer,brawl chess coming soon follow to play brawlchess retweet if you play chess chess chiesu szachy ...
53,GamesRedDeer,do you remember the times when highscore was what really mattered cyberprotocol77 is coming to s...
43,GamesRedDeer,brawl chess coming soon follow to play brawlchess retweet if you play chess chess chiesu szachy ...
133,GamesRedDeer,the end of the world is not the worst thing that can happen to us do you like space horrors we h...
148,GamesRedDeer,they say its one of the hardest logic game ever 100 levels global ranking try now nintendo xbox ...
348,GamesRedDeer,waiting for fallguys on nintendo try now joy of clumsyrush cosy party game fallguysgame nintendo...


#### Medios de comunicación.

In [94]:
df_medios = pd.read_csv('Data/Clasificados/usuarios_mas_10_tweets_clasificados.csv', sep=';')

In [95]:
mask_media = [x in usuarios_medios for x in df_medios['username']] 

In [96]:
df_medios.loc[mask_media, ['username','tweet']].sample(20)

,username,tweet
97,radiofonica1007,14 de agosto curvas de la evolucion del coronavirus en rosario santa fe y argentina radiofonica
483,gerdellamonica,reportan otros 3 casos de coronavirus en la provincia de santa fe 2 de la ciudad capital y 1 de ...
474,gustavorearte1,la oms afirma que no hay evidencia de que una persona que se haya recuperado del coronavirus no ...
464,gustavorearte1,un estudio publicado en the new england journal of medicine demuestra que un virus permanece has...
486,gerdellamonica,monte vera otra vez con 9 nuevos casos de coronavirus tremendo
473,gustavorearte1,argentina no tiene plata para auxiliar a su sector productivo entonces generalmente crea bonos a...
471,gustavorearte1,el ministro de salud bonaerense daniel gollan recomendo el uso de tapabocas para minimizar las p...
475,gustavorearte1,poniendo primera la automotriz scania reabre su fabrica de tucuman a partir del lunes retomara s...
478,gerdellamonica,la provincia de santa fe no registro casos de coronavirus
479,gerdellamonica,oficial 240 nuevos casos de coronavirus en argentina suman 5611 infectados 1659 recuperados 11 f...


### Por contenido de tweets.
#### Duplicados.

In [97]:
df[df.duplicated(subset=['tweet'])==True].sort_values(by='tweet',ascending=False)

,username,tweet,fecha,anuncio,ubicacion,id,sentimiento,fecha_sola
1676,Jefa04120216,zenonbiagosch steinbetina jorgecarreraok arnaldobocco alferdez bancocentralar automoruva alferde...,2020-06-04 18:53:55+00:00,Anuncio_7,"Mendoza, Argentina",1.268617e+18,NaN,NaN
1652,Jefa04120216,zenonbiagosch steinbetina jorgecarreraok arnaldobocco alferdez bancocentralar automoruva alferde...,2020-06-04 18:53:55+00:00,Anuncio_7,"Tucumán, Argentina",1.268617e+18,NaN,NaN
1653,Jefa04120216,zenonbiagosch steinbetina jorgecarreraok arnaldobocco alferdez bancocentralar automoruva alferde...,2020-06-04 18:53:55+00:00,Anuncio_7,"Paraná, Argentina",1.268617e+18,NaN,NaN
96,TeleCentroAyuda,yolavieja si despues del reseteo no recuperaste el servicio mandanos un mensaje de whats app al ...,2020-03-19 20:06:16+00:00,Anuncio_1,"Mendoza, Argentina",1.240731e+18,NaN,NaN
96,TeleCentroAyuda,yolavieja si despues del reseteo no recuperaste el servicio mandanos un mensaje de whats app al ...,2020-03-19 20:06:16+00:00,Anuncio_1,"Paraná, Argentina",1.240731e+18,NaN,NaN
...,...,...,...,...,...,...,...,...
2328,Rosana841773501,NaN,2020-07-17 21:20:16+00:00,Anuncio_9,"Paraná, Argentina",1.284237e+18,NaN,NaN
2338,Rosana841773501,NaN,2020-07-17 21:16:23+00:00,Anuncio_9,"Paraná, Argentina",1.284236e+18,NaN,NaN
2339,Rosana841773501,NaN,2020-07-17 21:16:00+00:00,Anuncio_9,"Paraná, Argentina",1.284235e+18,NaN,NaN
2363,Fernand98267834,NaN,2020-07-17 21:01:06+00:00,Anuncio_9,"Paraná, Argentina",1.284232e+18,NaN,NaN


#### Tópicos no relevantes.

In [98]:
print(df.shape)
df.dropna(subset=['tweet'], inplace=True)
print(df.shape)

(13857, 8)
(13469, 8)


<ipython-input-98-326fdc169b4b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(subset=['tweet'], inplace=True)


In [99]:
topicos_excluidos = ['pique','barcelona', 'bayern munich', 'barca', 'bayern', 'barsa', 'anabelle', 'annabell', 'annabelle', 'messi', 'balvin', 'pampita']

for t in topicos_excluidos:
    mask_topicos = df['tweet'].str.find(t) != -1
    
df.loc[mask_topicos, ['username','tweet']]

,username,tweet


### Guardamos el dataset filtrado.

In [24]:
df.to_csv("Data/Crudos/tweets_crudos_limpios_"+str(time.time())+".csv", sep=';', index=False)

## Merge de datasets.

Con el fin de sumar la mayor cantidad de tweets, es necesario que vayamos armando el dataframe consolidado con los datos que vamos obteniendo de "a cachos".

Por esta razón, tenemos tres datasets:
1. Nuevos tweets obtenidos sin clasificar.
2. Tweets clasificados.
3. Dataframe ya consolidado con todos los tweets obtenidos hasta el momento.

### Datasets con archivo consolidado.

In [25]:
try:
    list_of_files = glob.glob('Data/Consolidado/*.csv') 
    latest_file = max(list_of_files, key=os.path.getctime)
    df_consolidado_nuevo = pd.read_csv(latest_file, sep=';')
except:
    print("No hay archivos en el directorio")

In [26]:
df_consolidado_nuevo.head()

,username,tweet,fecha,anuncio,ubicacion,id,sentimiento,fecha_sola
0,Elizabeth789741,alferdez alferdezprensa por favor qxsea a part...,2020-03-19 23:04:48+00:00,Anuncio_1,"Buenos Aires, Argentina",1.240776e+18,NaN,NaN
1,sergistack,usas google chrome lo siento por las mayuscula...,2020-03-28 20:07:18+00:00,Anuncio_1,"Buenos Aires, Argentina",1.243993e+18,NaN,NaN
2,vickyuliyapo,comodice inesazpelicueta que comiencen ya a el...,2020-03-19 22:57:33+00:00,Anuncio_1,"Buenos Aires, Argentina",1.240774e+18,NaN,NaN
3,RUCHOCASLA,alferdez alferdezprensa a los que enganchan en...,2020-03-19 20:28:12+00:00,Anuncio_1,"Buenos Aires, Argentina",1.240737e+18,NaN,NaN
4,Populismomata,alferdezprensa porque no hacen que el indek mi...,2020-03-19 17:46:55+00:00,Anuncio_1,"Buenos Aires, Argentina",1.240696e+18,NaN,NaN


In [27]:
df_consolidado = clean_up_df(df_consolidado_nuevo)
print(df_consolidado.shape)
df_consolidado.head(3)

No existe la columna index
(7480, 7)


,username,tweet,fecha,anuncio,ubicacion,id,sentimiento
1199,Federic22912449,alferdezprensa fijense el mic por momentos no ...,2020-05-08 23:50:46+00:00,Anuncio_5,"Buenos Aires, Argentina",1.258907e+18,NaN
1496,DaniBrie1,esta foto es de ayer alferdez alferdezprensa s...,2020-05-23 18:30:48+00:00,Anuncio_6,"Buenos Aires, Argentina",1.264263e+18,NaN
1723,dany000037,alferdezprensa circula esto en el nombre de pr...,2020-06-04 20:23:27+00:00,Anuncio_7,"Buenos Aires, Argentina",1.268640e+18,NaN


### Datasets con archivos sin clasificar.

Obtenemos todos los archivos crudos limpios y los conformamos en un único dataframe:

In [28]:
path_crudos = 'Data/Crudos/' 
all_files_crudos = glob.glob(path_crudos + "/*.csv")

li_crudos = []

try:
    for filename in all_files_crudos:
        df_sin_clasificar = pd.read_csv(filename, sep = ';', header=0)
        li_crudos.append(df_sin_clasificar)
    concatenado = pd.concat(li_crudos, axis=0) 
    df_sin_clasificar = concatenado.drop_duplicates()
    print(df_sin_clasificar.shape)
except:
    print("No hay archivos en el directorio")

(13857, 8)


In [29]:
print(df_sin_clasificar.shape)
df_sin_clasificar.head(3)

(13857, 8)


,username,tweet,fecha,anuncio,ubicacion,id,sentimiento,fecha_sola
0,Elizabeth789741,alferdez alferdezprensa por favor qxsea a part...,2020-03-19 23:04:48+00:00,Anuncio_1,"Buenos Aires, Argentina",1.240776e+18,NaN,NaN
1,sergistack,usas google chrome lo siento por las mayuscula...,2020-03-28 20:07:18+00:00,Anuncio_1,"Buenos Aires, Argentina",1.243993e+18,NaN,NaN
2,vickyuliyapo,comodice inesazpelicueta que comiencen ya a el...,2020-03-19 22:57:33+00:00,Anuncio_1,"Buenos Aires, Argentina",1.240774e+18,NaN,NaN


### Dataset con tweets clasificados manualmente.

Obtenemos todos los archivos con tweets clasificados y los conformamos en un único dataframe:

In [30]:
path_clasificados = 'Data/Clasificados/' 
all_files_clasificados = glob.glob(path_clasificados + "/*.csv")

li_clasificados = []

try:
    for filename in all_files_clasificados:
        df_clasificados = pd.read_csv(filename, sep = ';', header=0)
        li_clasificados.append(df_clasificados)
        print(filename)
    concatenado = pd.concat(li_clasificados, axis=0) 
    df_clasificados = concatenado.drop_duplicates()
    print(df_clasificados.shape)
except:
    print("No hay archivos en el directorio")

Data/Clasificados\2da_mitad_tweets_clasificados_euge.csv
Data/Clasificados\parte_gaston.csv
Data/Clasificados\usuarios_mas_10_tweets_clasificados.csv
(5731, 8)


In [31]:
print(df_clasificados.shape)
df_clasificados.head(3)

(5731, 8)


,username,tweet,fecha,anuncio,ubicacion,fecha_sola,sentimiento,id
0,Mar-14,edufeiok edufeiok te comento tengo a mi hijo c...,2020-07-31 21:27:55+00:00,Anuncio_10,"Buenos Aires, Argentina",31-07-20,Neutral,NaN
1,____Marcos77,aguante el coronavirus,2020-08-14 00:30:36+00:00,Anuncio_11,"Cordoba, Argentina",14-08-20,Neutral,NaN
2,__Maxi94,mamita pique ni el coronavirus se agarra porqu...,2020-08-14 19:23:55+00:00,Anuncio_11,"Buenos Aires, Argentina",14-08-20,Neutral,NaN


In [32]:
df_clasificados.sentimiento.value_counts()

Neutral     1891
Positivo     487
Negativo     325
Name: sentimiento, dtype: int64

## Merge entre dataset clasificado y sin clasificar.

In [33]:
print('Cantidad de tweets clasificados en el dataset consolidado, previo al merge:', df_consolidado.sentimiento.notnull().sum())

Cantidad de tweets clasificados en el dataset consolidado, previo al merge: 2598


In [34]:
print('Cantidad de tweets clasificados en el dataset sin clasificar, previo al merge:', df_sin_clasificar.sentimiento.notnull().sum())

Cantidad de tweets clasificados en el dataset sin clasificar, previo al merge: 0


In [35]:
print('Cantidad de tweets clasificados en el dataset clasificados, previo al merge:', df_clasificados.sentimiento.notnull().sum())

Cantidad de tweets clasificados en el dataset clasificados, previo al merge: 2703


In [36]:
df_appended = df_sin_clasificar.append(df_clasificados)
print(df_appended.shape)
df_appended.head(3)

(19588, 8)


,username,tweet,fecha,anuncio,ubicacion,id,sentimiento,fecha_sola
0,Elizabeth789741,alferdez alferdezprensa por favor qxsea a part...,2020-03-19 23:04:48+00:00,Anuncio_1,"Buenos Aires, Argentina",1.240776e+18,NaN,NaN
1,sergistack,usas google chrome lo siento por las mayuscula...,2020-03-28 20:07:18+00:00,Anuncio_1,"Buenos Aires, Argentina",1.243993e+18,NaN,NaN
2,vickyuliyapo,comodice inesazpelicueta que comiencen ya a el...,2020-03-19 22:57:33+00:00,Anuncio_1,"Buenos Aires, Argentina",1.240774e+18,NaN,NaN


In [37]:
df_consolidado = df_appended.drop_duplicates()
df_consolidado.shape

(19588, 8)

In [38]:
df_consolidado.describe()

,id
count,1.458100e+04
mean,1.263006e+18
std,1.820812e+16
min,1.213622e+18
25%,1.244364e+18
50%,1.264161e+18
75%,1.276653e+18
max,1.300160e+18


Revisamos a cuántos tweets se les impactó el valor de `sentimiento`:

In [39]:
print('Cantidad de tweets clasificados en el dataset, luego del merge:', df_consolidado.sentimiento.notnull().sum())

Cantidad de tweets clasificados en el dataset, luego del merge: 2703


In [40]:
df_consolidado.sentimiento.value_counts()

Neutral     1891
Positivo     487
Negativo     325
Name: sentimiento, dtype: int64

In [41]:
df = df_consolidado
df.shape

(19588, 8)